In [25]:
import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.python.data import Dataset

import numpy as np

In [26]:
mnist = load_data("mnist.npz")
trainX, trainY = np.reshape(mnist[0][0],[-1,784]), mnist[0][1].astype(np.int32)
testX, testY = np.reshape(mnist[1][0],[-1,784]), mnist[1][1].astype(np.int32)

In [33]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    ds = Dataset.from_tensor_slices((features,targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
        ds = ds.shuffle(10000)
    
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [28]:
def construct_feature_columns(input_features):
    return set([tf.feature_column.numeric_column(my_feature,shape=[784]) for my_feature in input_features])

In [56]:
def train_nn_model(learning_rate,steps,batch_size,hidden_units,training_examples,training_targets,validation_examples,validation_targets):   
    my_optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    estimator = tf.estimator.DNNClassifier(
        feature_columns=construct_feature_columns(training_examples),
        hidden_units=hidden_units,
        n_classes=10,
        model_dir="./model/estimator",
        optimizer=my_optimizer
    )
    
    training_input_fn = lambda: my_input_fn(training_examples, training_targets, batch_size=batch_size, shuffle=False)
    validation_input_fn = lambda: my_input_fn(validation_examples, validation_targets, shuffle=False, num_epochs=1)

    estimator.train(input_fn=training_input_fn, steps=steps,)
    accuracy_score = estimator.evaluate(input_fn=validation_input_fn)["accuracy"]
    
    print("Test accuracy is {}".format(accuracy_score))    

In [57]:
train_nn_model(0.005, 6000, 100, [500], {"x":trainX}, trainY, {"x":testX}, testY)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.

Test accuracy is 0.9391000270843506
